In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Use ChromeDriverManager to automatically handle driver setup
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Navigate to the CoinMarketCap historical page for January 6, 2019
url = 'https://coinmarketcap.com/historical/20210103/'
driver.get(url)

# Allow initial load
time.sleep(5)

# Initialize data storage
data = []
assets_loaded = 0

# Continue scrolling until we capture 125 assets
while assets_loaded < 125:
    try:
        # Locate the table rows containing assets data
        table_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
        
        # Extract data for each asset if new rows are loaded
        if len(table_rows) > assets_loaded:
            for row in table_rows[assets_loaded:]:
                try:
                    columns = row.find_elements(By.TAG_NAME, "td")
                    if len(columns) >= 10:  # Ensure row has the expected number of columns
                        rank = columns[0].text
                        name = columns[1].text
                        symbol = columns[2].text
                        market_cap = columns[3].text
                        price = columns[4].text
                        circulating_supply = columns[5].text
                        volume_24h = columns[6].text
                        change_1h = columns[7].text
                        change_24h = columns[8].text
                        change_7d = columns[9].text

                        # Append data for the asset
                        data.append([rank, name, symbol, market_cap, price, circulating_supply, volume_24h, change_1h, change_24h, change_7d])
                        assets_loaded += 1

                        # Stop if we've captured 125 assets
                        if assets_loaded >= 125:
                            break
                except Exception as e:
                    print(f"Error extracting row data: {e}")
        else:
            # Scroll down and wait for the new assets to load
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
    except Exception as e:
        print(f"Error locating rows: {e}")
        # Re-scroll to try again
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

# Close the browser
driver.quit()

# Create a DataFrame with the data
columns = ["Rank", "Name", "Symbol", "Market Cap", "Price", "Circulating Supply", "Volume (24h)", "% 1h", "% 24h", "% 7d"]
df = pd.DataFrame(data, columns=columns)

# Display the DataFrame (or save to a file if needed)
print(df)

# Optional: Save to CSV
df.to_csv("coinmarketcap_20210103_top_125.csv", index=False)

WebDriverException: Message: Service /Users/maziyarmahmoodi/.wdm/drivers/chromedriver/mac64/131.0.6778.69/chromedriver-mac-arm64/chromedriver unexpectedly exited. Status code was: -9
